# VQE Optimization Tracking with Qiskit (devqubit demo)

This notebook is a **minimal, reproducible** example of running a small VQE-style optimization loop in Qiskit **while devqubit tracks everything you need to debug and compare runs**:

- step-by-step metrics (energy, best-so-far, parameter norm)
- captured circuits / artifacts via backend wrapping
- grouped sweeps (compare optimizer hyperparameters)
- baseline + verification (catch regressions)
- a shareable bundle of a full run (params, metrics, artifacts)

In [ ]:
from __future__ import annotations

from importlib.metadata import entry_points


def has_adapter(name: str) -> bool:
    eps = entry_points().select(group="devqubit.adapters")
    return any(ep.name == name for ep in eps)


if not has_adapter("qiskit"):
    raise ImportError(
        "devqubit Qiskit adapter is not installed.\n"
        "Install with: pip install 'devqubit[qiskit]'"
    )

print("Qiskit adapter available!")

In [ ]:
from dataclasses import asdict, dataclass
from pathlib import Path
from typing import Dict, List, Tuple
import shutil

import numpy as np

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit_aer import AerSimulator

from devqubit import (
    diff,
    create_registry,
    create_store,
    pack_run,
    track,
    wrap_backend,
    verify,
)
from devqubit.compare import VerifyPolicy

In [ ]:
"""Configuration."""

# -----------------------------------------------------------------------------
# Global configuration
# -----------------------------------------------------------------------------
PROJECT_NAME: str = "vqe_ising"
DEFAULT_SEED: int = 42

# Workspace
WORKSPACE_DIR: Path = Path(".devqubit_vqe_demo")

# Problem size
N_QUBITS: int = 4
N_LAYERS: int = 3

# Shot budgets
TRAIN_SHOTS: int = 2_048
VALIDATION_SHOTS: int = 8_192

# Optimization (simple stochastic hill-climb)
N_STEPS: int = 50
LR_INIT: float = 0.1
LR_DECAY: float = 0.95
PERTURB_SCALE: float = 1.0  # stddev = lr * PERTURB_SCALE
EVAL_EVERY: int = 5  # compute validation energy every N steps

# “Exact” reference for the toy Hamiltonian
EXACT_GROUND_ENERGY: float = -(N_QUBITS - 1)


@dataclass(frozen=True)
class VQERunConfig:
    """Configuration for one tracked VQE-style run."""

    n_qubits: int
    n_layers: int
    n_steps: int
    train_shots: int
    val_shots: int
    lr_init: float
    lr_decay: float
    perturb_scale: float = 1.0
    eval_every: int = 5
    seed: int = DEFAULT_SEED
    optimizer_name: str = "stochastic_hillclimb"


def reset_workspace(path: Path) -> None:
    """Create a clean local workspace directory."""

    if path.exists():
        shutil.rmtree(path)
    path.mkdir(parents=True, exist_ok=True)


reset_workspace(WORKSPACE_DIR)

store = create_store(f"file://{WORKSPACE_DIR}/objects")
registry = create_registry(f"file://{WORKSPACE_DIR}")

print(f"Workspace: {WORKSPACE_DIR.resolve()}")

---
## 1. Define the toy VQE problem

We use a chain of **4 qubits** with the Hamiltonian

$H = - \sum_{i=0}^{n-2} Z_i Z_{i+1}$

For this model the exact ground-state energy is $E_0 = -(n-1)$.

Because this Hamiltonian is diagonal in the computational basis, we can estimate its energy
from **measurement counts** only (no basis changes needed).

We will:
1. build a small hardware-efficient ansatz
2. estimate energy from counts
3. run a simple optimization loop (stochastic hill-climb)
4. let devqubit track metrics + circuits along the way


In [ ]:
"""Circuit + Hamiltonian utilities."""


def build_hardware_efficient_ansatz(
    n_qubits: int,
    n_layers: int,
) -> Tuple[QuantumCircuit, List[Parameter]]:
    """Build a small hardware-efficient ansatz circuit."""

    parameters: List[Parameter] = []
    qc = QuantumCircuit(n_qubits, name=f"hea_L{n_layers}")

    for layer in range(n_layers):
        # Rotation layer
        for q in range(n_qubits):
            p = Parameter(f"theta_{layer}_{q}")
            parameters.append(p)
            qc.ry(p, q)

        # Entanglement layer (chain)
        for q in range(n_qubits - 1):
            qc.cx(q, q + 1)

    # Shot-based energy uses measurement counts
    qc.measure_all()
    return qc, parameters


def ising_energy_from_counts(counts: Dict[str, int], n_qubits: int) -> float:
    """
    Estimate the Ising energy from measurement counts.

    Notes
    -----
    Qiskit count keys are bitstrings where **qubit 0 is the rightmost bit**.
    We reverse the bitstring so index `i` corresponds to qubit `i`.
    """
    total = sum(counts.values())
    if total == 0:
        raise ValueError("Counts are empty; cannot estimate energy.")

    energy_acc = 0.0
    for bitstring, count in counts.items():
        bits = bitstring[::-1]  # map to qubit order
        spins = np.array([1 if b == "0" else -1 for b in bits[:n_qubits]], dtype=int)

        # Energy contribution for this sample: -Σ s_i s_{i+1}
        e_sample = 0
        for i in range(n_qubits - 1):
            e_sample -= spins[i] * spins[i + 1]

        energy_acc += e_sample * count

    return float(energy_acc) / float(total)


def bind_parameters(
    circuit: QuantumCircuit,
    parameters: List[Parameter],
    values: np.ndarray,
) -> QuantumCircuit:
    """Bind a numeric parameter vector into a parameterized circuit."""

    if len(parameters) != len(values):
        raise ValueError(
            f"Expected {len(parameters)} parameters, got {len(values)} values."
        )

    mapping = {p: float(v) for p, v in zip(parameters, values)}
    return circuit.assign_parameters(mapping, inplace=False)


def estimate_ising_energy(
    backend,
    circuit: QuantumCircuit,
    parameters: List[Parameter],
    values: np.ndarray,
    shots: int,
) -> float:
    """Run the circuit and estimate Ising energy from sampled counts."""

    bound = bind_parameters(circuit, parameters, values)
    counts = backend.run(bound, shots=int(shots)).result().get_counts()
    return ising_energy_from_counts(counts, n_qubits=len(circuit.qubits))


@dataclass
class VQETrace:
    """Container for VQE optimization results.

    Attributes
    ----------
    theta_best
        Best parameter vector found (according to training energy).
    best_train_energy
        Best (lowest) training energy encountered.
    best_val_energy
        Validation energy of `theta_best` at the time it was recorded.
    train_energy_history
        Per-step training energy values.
    val_energy_history
        Per-evaluation validation energy values (sparser than training history).
    """

    theta_best: np.ndarray
    best_train_energy: float
    best_val_energy: float
    train_energy_history: List[float]
    val_energy_history: List[float]


def stochastic_hillclimb_vqe(
    backend,
    circuit: QuantumCircuit,
    parameters: List[Parameter],
    config: VQERunConfig,
    log_step_fn=None,
) -> VQETrace:
    """
    A tiny, noisy, VQE-style optimizer (stochastic hill-climb).

    This is intentionally simple:
    1. start from random parameters
    2. propose a random perturbation
    3. accept if energy improves
    4. decay step-size over time
    """
    rng = np.random.default_rng(config.seed)

    theta = rng.uniform(0.0, 2.0 * np.pi, size=len(parameters))
    lr = float(config.lr_init)

    best_train_energy = float("inf")
    best_val_energy = float("inf")
    theta_best = theta.copy()

    train_history: List[float] = []
    val_history: List[float] = []

    for step in range(config.n_steps):
        # Evaluate current point (training shot budget)
        energy = estimate_ising_energy(
            backend=backend,
            circuit=circuit,
            parameters=parameters,
            values=theta,
            shots=config.train_shots,
        )
        train_history.append(float(energy))

        if energy < best_train_energy:
            best_train_energy = float(energy)
            theta_best = theta.copy()

        metrics = {
            "train_energy": float(energy),
            "best_train_energy": float(best_train_energy),
            "lr": float(lr),
            "param_norm": float(np.linalg.norm(theta)),
        }

        # Periodic validation re-evaluation (more shots)
        if config.eval_every and (step % config.eval_every == 0):
            val_energy = estimate_ising_energy(
                backend=backend,
                circuit=circuit,
                parameters=parameters,
                values=theta_best,
                shots=config.val_shots,
            )
            best_val_energy = float(val_energy)
            val_history.append(float(val_energy))
            metrics["val_energy"] = float(val_energy)

        if log_step_fn is not None:
            log_step_fn(step, metrics)

        # Propose & accept/reject
        proposal = rng.normal(
            loc=0.0, scale=lr * config.perturb_scale, size=len(parameters)
        )
        theta_trial = theta + proposal

        energy_trial = estimate_ising_energy(
            backend=backend,
            circuit=circuit,
            parameters=parameters,
            values=theta_trial,
            shots=config.train_shots,
        )

        if energy_trial < energy:
            theta = theta_trial

        lr *= float(config.lr_decay)

    return VQETrace(
        theta_best=theta_best,
        best_train_energy=float(best_train_energy),
        best_val_energy=float(best_val_energy),
        train_energy_history=train_history,
        val_energy_history=val_history,
    )


# Build the problem instance used throughout the notebook
ansatz_circuit, ansatz_params = build_hardware_efficient_ansatz(N_QUBITS, N_LAYERS)

print(f"Ansatz parameters: {len(ansatz_params)}")
print(f"Exact ground energy: {EXACT_GROUND_ENERGY}")
print("\nCircuit diagram:")
print(ansatz_circuit.draw(fold=500))

---
## 2. Run a single baseline VQE-style optimization (fully tracked)

devqubit will log:
- per-step training energy (shot-noisy)
- best-so-far energy
- occasional validation energy (more shots)
- parameter norm and learning-rate schedule
- the executed circuits (via backend wrapping)


In [ ]:
"""Run one baseline optimization with step-wise tracking."""

baseline_config = VQERunConfig(
    n_qubits=N_QUBITS,
    n_layers=N_LAYERS,
    n_steps=N_STEPS,
    train_shots=TRAIN_SHOTS,
    val_shots=VALIDATION_SHOTS,
    lr_init=LR_INIT,
    lr_decay=LR_DECAY,
    perturb_scale=PERTURB_SCALE,
    eval_every=EVAL_EVERY,
    seed=DEFAULT_SEED,
    optimizer_name="stochastic_hillclimb",
)

with track(project=PROJECT_NAME, store=store, registry=registry) as run:
    # Wrap the backend so devqubit captures circuits + artifacts automatically.
    backend = wrap_backend(run, AerSimulator(seed_simulator=baseline_config.seed))

    # Record the full configuration (no magic numbers later).
    run.log_params(
        {**asdict(baseline_config), "exact_ground_energy": EXACT_GROUND_ENERGY}
    )
    run.set_tags(
        {
            "algorithm": "VQE",
            "hamiltonian": "ising_1d",
            "status": "baseline",
        }
    )

    def log_step(step: int, metrics: dict) -> None:
        """Log per-step metrics into devqubit."""
        run.log_metric("train_energy", metrics["train_energy"], step=step)
        run.log_metric("best_train_energy", metrics["best_train_energy"], step=step)
        run.log_metric("param_norm", metrics["param_norm"], step=step)
        run.log_metric("lr", metrics["lr"], step=step)
        if "val_energy" in metrics:
            run.log_metric("val_energy", metrics["val_energy"], step=step)

    trace = stochastic_hillclimb_vqe(
        backend=backend,
        circuit=ansatz_circuit,
        parameters=ansatz_params,
        config=baseline_config,
        log_step_fn=log_step,
    )

    # Summary metrics (easy to compare across runs)
    run.log_metrics(
        {
            "final_train_energy": trace.train_energy_history[-1],
            "best_train_energy": trace.best_train_energy,
            "best_val_energy": trace.best_val_energy,
            "energy_error_vs_exact": float(trace.best_val_energy - EXACT_GROUND_ENERGY),
        }
    )

    # Save best parameters for reproducibility / later reuse.
    run.log_json(
        name="best_theta",
        obj={"theta": trace.theta_best.tolist()},
        role="results",
    )

    baseline_id = run.run_id

print(f"Run ID: {baseline_id}")
print(f"Best train energy: {trace.best_train_energy:.4f}")
print(f"Best val energy: {trace.best_val_energy:.4f}")
print(f"Exact ground energy: {EXACT_GROUND_ENERGY:.4f}")

**VQE results:** the best validation energy should drift toward the exact ground energy as the optimizer finds better parameters. With finite shots, you may still see noise.

---
## 3. Compare optimizer hyperparameters with a grouped sweep

devqubit’s `group_id` lets us run a small sweep (same initialization seed) and compare runs cleanly.
We pick the “winner” by **best validation energy** (less sensitive to shot noise).


In [ ]:
"""Hyperparameter sweep (grouped runs)."""

sweep_id = "optimizer_sweep_example"
print(f"Optimizer sweep group_id: {sweep_id}")
print("-" * 60)

sweep_configs = [
    VQERunConfig(
        n_qubits=N_QUBITS,
        n_layers=N_LAYERS,
        n_steps=N_STEPS,
        train_shots=TRAIN_SHOTS,
        val_shots=VALIDATION_SHOTS,
        lr_init=0.10,
        lr_decay=0.98,
        perturb_scale=PERTURB_SCALE,
        eval_every=EVAL_EVERY,
        seed=DEFAULT_SEED,  # same init for fair comparison
        optimizer_name="search_lr0.10_decay0.98",
    ),
    VQERunConfig(
        n_qubits=N_QUBITS,
        n_layers=N_LAYERS,
        n_steps=N_STEPS,
        train_shots=TRAIN_SHOTS,
        val_shots=VALIDATION_SHOTS,
        lr_init=0.30,
        lr_decay=0.95,
        perturb_scale=PERTURB_SCALE,
        eval_every=EVAL_EVERY,
        seed=DEFAULT_SEED,
        optimizer_name="search_lr0.30_decay0.95",
    ),
    VQERunConfig(
        n_qubits=N_QUBITS,
        n_layers=N_LAYERS,
        n_steps=N_STEPS,
        train_shots=TRAIN_SHOTS,
        val_shots=VALIDATION_SHOTS,
        lr_init=0.50,
        lr_decay=0.90,
        perturb_scale=PERTURB_SCALE,
        eval_every=EVAL_EVERY,
        seed=DEFAULT_SEED,
        optimizer_name="search_lr0.50_decay0.90",
    ),
]

sweep_results = []

for cfg in sweep_configs:
    with track(
        project=PROJECT_NAME,
        store=store,
        registry=registry,
        group_id=sweep_id,
        group_name="Optimizer Comparison",
    ) as run:
        backend = wrap_backend(run, AerSimulator(seed_simulator=cfg.seed))

        run.log_params(
            {
                **asdict(cfg),
                "exact_ground_energy": EXACT_GROUND_ENERGY,
            }
        )
        run.set_tags(
            {
                "algorithm": "VQE",
                "hamiltonian": "ising_1d",
                "status": "sweep",
            }
        )
        run.set_tag("sweep", "optimizer")

        def log_step(step: int, metrics: dict) -> None:
            run.log_metric(
                "train_energy",
                metrics["train_energy"],
                step=step,
            )
            run.log_metric(
                "best_train_energy",
                metrics["best_train_energy"],
                step=step,
            )
            if "val_energy" in metrics:
                run.log_metric(
                    "val_energy",
                    metrics["val_energy"],
                    step=step,
                )

        trace = stochastic_hillclimb_vqe(
            backend=backend,
            circuit=ansatz_circuit,
            parameters=ansatz_params,
            config=cfg,
            log_step_fn=log_step,
        )

        run.log_metrics(
            {
                "best_train_energy": trace.best_train_energy,
                "best_val_energy": trace.best_val_energy,
                "energy_error_vs_exact": float(
                    trace.best_val_energy - EXACT_GROUND_ENERGY
                ),
            }
        )

        sweep_results.append(
            {
                "optimizer": cfg.optimizer_name,
                "best_val_energy": trace.best_val_energy,
                "run_id": run.run_id,
            }
        )

    print(f"  {cfg.optimizer_name:22s}  best_val_energy={trace.best_val_energy:+.4f}")

winner = min(sweep_results, key=lambda r: r["best_val_energy"])
print("\nWinner:")
print(
    f"  {winner['optimizer']}  E_val={winner['best_val_energy']:+.4f}  run_id={winner['run_id'][:12]}..."
)

---
## 4. Set a baseline run

Once you have a run you trust (e.g., your current best config), set it as a **baseline** so future runs can be verified against it.


In [ ]:
"""Mark the baseline run in the registry."""

registry.set_baseline(PROJECT_NAME, baseline_id)
print(f"Baseline set for {PROJECT_NAME}: {baseline_id[:20]}...")

In [ ]:
"""Run a candidate run and verify it against the stored baseline."""

candidate_config = VQERunConfig(
    n_qubits=N_QUBITS,
    n_layers=N_LAYERS,
    n_steps=N_STEPS,
    train_shots=TRAIN_SHOTS,
    val_shots=VALIDATION_SHOTS,
    lr_init=LR_INIT,
    lr_decay=LR_DECAY,
    perturb_scale=PERTURB_SCALE,
    eval_every=EVAL_EVERY,
    seed=DEFAULT_SEED,
    optimizer_name="stochastic_hillclimb",
)

with track(
    project=PROJECT_NAME,
    store=store,
    registry=registry,
) as run:
    backend = wrap_backend(run, AerSimulator(seed_simulator=candidate_config.seed))

    run.log_params(
        {
            **asdict(candidate_config),
            "exact_ground_energy": EXACT_GROUND_ENERGY,
        }
    )
    run.set_tags(
        {
            "algorithm": "VQE",
            "hamiltonian": "ising_1d",
            "status": "candidate",
        }
    )

    def log_step(step: int, metrics: dict) -> None:
        run.log_metric(
            "train_energy",
            metrics["train_energy"],
            step=step,
        )
        run.log_metric(
            "best_train_energy",
            metrics["best_train_energy"],
            step=step,
        )
        if "val_energy" in metrics:
            run.log_metric(
                "val_energy",
                metrics["val_energy"],
                step=step,
            )

    trace_candidate = stochastic_hillclimb_vqe(
        backend=backend,
        circuit=ansatz_circuit,
        parameters=ansatz_params,
        config=candidate_config,
        log_step_fn=log_step,
    )

    run.log_metrics(
        {
            "best_train_energy": trace_candidate.best_train_energy,
            "best_val_energy": trace_candidate.best_val_energy,
        }
    )
    run.log_json(
        name="best_theta",
        obj={"theta": trace_candidate.theta_best.tolist()},
        role="results",
    )

    candidate_id = run.run_id

# ---------------------------------------------------------------------------
# Verification
# ---------------------------------------------------------------------------
baseline_rec = registry.load(baseline_id)
candidate_rec = registry.load(candidate_id)

policy = VerifyPolicy(
    params_must_match=True,
    program_must_match=True,
    fingerprint_must_match=False,  # allow statistical variation
    tvd_max=0.15,  # tolerance for distribution distance
)

verification = verify(
    baseline_rec,
    candidate_rec,
    store_baseline=store,
    store_candidate=store,
    policy=policy,
)

print(f"Candidate run: {candidate_id[:12]}...")
print(f"Verification: {'PASSED' if verification.ok else 'FAILED'}")
if not verification.ok:
    for failure in verification.failures:
        print(f"  - {failure}")

---
## 5. Compare baseline to candidate

devqubit can produce a readable diff between two runs (metadata + artifacts). This is handy in PR reviews and debugging sessions.


In [ ]:
"""Human-readable comparison between two runs."""

comparison = diff(
    baseline_id,
    candidate_id,
    registry=registry,
    store=store,
)

print(comparison)

**Comparison analysis:** use this view when a candidate run regresses — you can quickly see whether *params*, *program*, or *output statistics* changed.

---
## 6. List and summarize all runs

A tiny “run table” helps you scan which optimizer configs produced the lowest energies.


In [ ]:
"""Query and summarize all runs in this project."""

runs = registry.list_runs(project=PROJECT_NAME)

print("VQE Run Summary")
print("=" * 80)
print(f"Exact ground energy: {EXACT_GROUND_ENERGY:+.3f}\n")

for info in runs:
    rec = registry.load(info["run_id"])

    opt = str(rec.params.get("optimizer_name", rec.params.get("optimizer", "N/A")))
    status = str(rec.tags.get("status", ""))
    best_val = rec.metrics.get("best_val_energy", None)
    best_train = rec.metrics.get(
        "best_train_energy", rec.metrics.get("best_energy", None)
    )

    def fmt(x):
        return "N/A" if x is None else f"{float(x):+.3f}"

    quality = ""
    if best_val is not None:
        quality = "✓" if float(best_val) < (EXACT_GROUND_ENERGY + 0.5) else "✗"

    print(
        f"  {rec.run_id[:12]}...  "
        f"{opt:24s}  "
        f"E_val={fmt(best_val)}  E_train={fmt(best_train)}  {quality}  [{status}]"
    )

---
## 7. Bundle a run for sharing

devqubit can export a run (metrics, parameters, artifacts) into a single file you can share with collaborators
or attach to an issue/PR.


In [ ]:
"""Pack the baseline run into a shareable bundle."""

bundle_path = WORKSPACE_DIR / "vqe_baseline.devqubit.zip"

result = pack_run(
    run_id=baseline_id,
    output_path=bundle_path,
    store=store,
    registry=registry,
)

print(f"Bundle: {bundle_path.name}")
print(f"  Size:      {bundle_path.stat().st_size / 1024:.1f} KB")
print(f"  Artifacts: {result.artifact_count}")

**Bundle contents:** includes run metadata (params/tags), tracked metrics, and captured artifacts (e.g., circuits executed on the backend).

---
## Summary

In a compact VQE-style loop we demonstrated how devqubit helps you:

- track convergence step-by-step (train + validation energy)
- compare hyperparameter variants via grouped sweeps
- lock in a baseline and verify future candidates
- export a complete run bundle for sharing/review

This is intentionally a small example — the same workflow scales to larger ansätze,
more realistic Hamiltonians, and real hardware backends.


In [ ]:
# Optional cleanup (keeps reruns tidy).
# If you'd like to inspect the registry/artifacts on disk, comment this out.
shutil.rmtree(WORKSPACE_DIR)
print("Workspace cleaned up.")